In [1]:
import sys
sys.path.insert(0, "../../")
from pyecharts import online
online()
#  演示必要的准备代码，使用该库时不需重复此单元格命令

In [2]:
import xalpha as xa
import pandas as pd

# 策略制定类
生成简单的策略投资交易单，并储存在 self.status 中

### 一次性买入并持有

In [3]:
jshs = xa.fundinfo('000311')
# 选择投资标的

In [13]:
bah = xa.policy.buyandhold(jshs, start='2017-01-01', totmoney=100000)
# 简单的一次性买入类，从 start 日买入后就一直持有，始终选择分红再投入

In [5]:
bah.status
# 第二三列表示此两日恰为分红日，选择了分红再投入

,000311,date
0,100000.00,2017-01-03
1,0.05,2017-08-04
2,0.05,2017-08-15


In [6]:
jshs.fenhongdate
# 验证该两日确实分红

[Timestamp('2017-08-04 00:00:00'), Timestamp('2017-08-15 00:00:00')]

In [14]:
bah.sellout('2018-06-01') # 选定日期全部卖出

In [15]:
bah.status

,000311,date
0,100000.000,2017-01-03
1,0.050,2017-08-04
2,0.050,2017-08-15
3,-0.005,2018-06-01


In [16]:
jshstrade = xa.trade(jshs, bah.status) # 尝试交易一下

In [20]:
jshstrade.xirrrate()

0.15629726951248488

### 定投类

In [35]:
# 依旧先制定标的
zzcm = xa.fundinfo('164818')
zzcm

工银中证传媒指数分级

In [36]:
auto = xa.policy.scheduled(zzcm, 1000, pd.date_range('2015-07-01','2018-07-01',freq='W-THU'))
# 按照每周四定额定投1000，生成交易单

In [38]:
# 按上面的策略单子交易下
cm_t3 = xa.trade(zzcm, auto.status)

In [40]:
cm_t3.v_tradevolume() # 交易情况可视化，那些明显的缝隙对应了小长假等，当次定投下单会被自动延后到下一个交易日

In [41]:
cm_t3.dailyreport('2018-08-03')  ## 截止到2018.08.03， 定投已经浮亏了 40% 多

{'currentshare': 96030.82,
 'currentvalue': 89788.82,
 'date': Timestamp('2018-08-03 00:00:00'),
 'originalvalue': 155000.0,
 'returnrate': -42.0717,
 'unitcost': 1.6141,
 'unitvalue': 0.93500000000000005}

In [42]:
cm_t3.dailyreport('2018-08-03')['returnrate']

-42.0717

还可以根据指数位置设置定期不定额的定投策略生成，这里就不仔细展开，只给一个例子：

In [47]:
auto2 = xa.policy.scheduled_tune(zzcm, 1000, pd.date_range('2015-07-01','2018-07-01',freq='M'),
                                [(0.9,2),(1.2,1)]) # 净值0.9以下加倍定投，1.2以上不再定投，中间正常定投

In [50]:
cm_t4 = xa.trade(zzcm, auto2.status)

In [51]:
cm_t4.v_tradevolume()